In [1]:
from vllm import LLM, SamplingParams
import os
import sys
os.environ["VLLM_ATTENTION_BACKEND"] = "FLASHINFER"

llm = LLM(
    model="google/gemma-2-9b-it", 
)

WARNING 07-11 12:43:24 utils.py:562] Gemma 2 uses sliding window attention for every odd layer, which is currently not supported by vLLM. Disabling sliding window and capping the max length to the sliding window size (4096).
INFO 07-11 12:43:24 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=goog

In [2]:
import os
import sys
from pathlib import Path
module_path = Path(os.path.abspath("")).parent
print(module_path)
sys.path.append(str(module_path))

import pandas as pd
from prompts import PROMPTS
from tqdm import tqdm
reviews = pd.read_csv("../../data/reviewer2_ARR_2022_reviews.csv")
tokenizer = llm.get_tokenizer()
sampling_params = SamplingParams(
    temperature=0.0, top_p=1, max_tokens=32,
    stop_token_ids=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
)


/fsx/homes/Abdelrahman.Sadallah@mbzuai.ac.ae/mbzuai/peerq-generation/notebooks


In [3]:

aspects = ['Actionability','Constructiveness or Politeness','Credibility or Verifiability','Specificity']
for i, row in tqdm(reviews.iterrows(), total=reviews.shape[0]):
    review = row['focused_review']


    for aspect in aspects:
        aspect_desc = PROMPTS[aspect]
        prompt = PROMPTS['base_prompt'].format(aspect=aspect, aspect_description =aspect_desc, review=review)

        conversation = [
                        {
                            'role': 'user', 'content': prompt
                        }
                    ]
        c = tokenizer.apply_chat_template(
                    conversation,
                    tokenize=False,
                    add_generation_prompt=True
                )
        inputs = [c]
        outputs = llm.generate(inputs, sampling_params)[0].outputs[0].text.strip()

        reviews.at[i, aspect] = outputs

0it [00:00, ?it/s]
Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.81it/s, est. speed input: 2291.82 toks/s, output: 34.31 toks/s]

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.91it/s, est. speed input: 2492.85 toks/s, output: 34.81 toks/s]

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.43it/s, est. speed input: 2544.42 toks/s, output: 32.37 toks/s]

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.92it/s, est. speed input: 2350.93 toks/s, output: 34.88 toks/s]
0it [00:00, ?it/s]


In [4]:
reviews

,paped_id,paper_summary,summary_of_strengths,summary_of_weaknesses,"comments,_suggestions_and_typos",score_overall,score_best_paper,score_datasets,score_software,author_identity_guess,confidence,score_replicability,ethical_concerns,focused_review,Actionability,Constructiveness or Politeness,Credibility or Verifiability,Specificity
0,ARR_2022_20_review,This paper aims to find the best strategy to s...,Comprehensive empirical studies. Experiments s...,Although the experiments are very comprehensiv...,Can you provide more insights about why loss e...,3 = Good: This paper makes a reasonable contri...,No,1 = No usable datasets submitted.,3 = Potentially useful: Someone might find the...,1 = I do not have even an educated guess about...,"2 = Willing to defend my evaluation, but it i...",NaN,NaN,Although the experiments are very comprehensiv...,3 \n,3 \n,3 \n,4 \n
1,ARR_2022_99_review,This paper investigates whether self-attention...,- The paper is well written and easy to follow...,- The paper is not so clear as the introductio...,Comments: 1. If a model correlates better wi...,3.5,No,1 = No usable datasets submitted.,3 = Potentially useful: Someone might find the...,1 = I do not have even an educated guess about...,4 = Quite sure. I tried to check the important...,"4 = They could mostly reproduce the results, b...",NaN,- The paper is not so clear as the introductio...,NaN,NaN,NaN,NaN
2,ARR_2022_99_review,This paper compared the self-attention functio...,This paper focused on investigating the relati...,"I like this paper overall, but there are sever...",see above,3.5,No,1 = No usable datasets submitted.,1 = No usable software released.,1 = I do not have even an educated guess about...,4 = Quite sure. I tried to check the important...,3 = They could reproduce the results with some...,NaN,NaN,NaN,NaN,NaN,NaN
3,ARR_2022_241_review,This work evaluates 4 different ASR models und...,- The scenario investigated is indeed very use...,- the main contribution of fine-tuning a multi...,- One point I would like to read more about is...,"2 = Borderline: This paper has some merit, but...",No,4 = Useful: I would recommend the new datasets...,4 = Useful: I would recommend the new software...,3 = From the contents of the submission itself...,"3 = Pretty sure, but there's a chance I misse...",5 = They could easily reproduce the results.,NaN,- the main contribution of fine-tuning a multi...,NaN,NaN,NaN,NaN
4,ARR_2022_241_review,This paper is about the design of an automatic...,The strength of this paper is that it uses a u...,"The motivation, purpose, and selection of mode...",Even if I take into account the convenience of...,"2 = Borderline: This paper has some merit, but...",No,5 = Enabling: The newly released datasets shou...,5 = Enabling: The newly released software shou...,1 = I do not have even an educated guess about...,4 = Quite sure. I tried to check the important...,"4 = They could mostly reproduce the results, b...",NaN,"The motivation, purpose, and selection of mode...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,ARR_2022_230_review,A comprehensive set of experiments are conduct...,- A comprehensive comparison of models on text...,- Limited to multi-class topical text categori...,- Can parameter counts/training time for graph...,4 = Strong: This paper is of significant inter...,No,1 = No usable datasets submitted.,4 = Useful: I would recommend the new software...,1 = I do not have even an educated guess about...,4 = Quite sure. I tried to check the important...,"4 = They could mostly reproduce the results, b...",NaN,- Limited to multi-class topical text categori...,NaN,NaN,NaN,NaN
680,ARR_2022_125_review,In biaffine graph-based architecture for seman...,- The motivation of adding the auxiliary tasks...,- It seems that how to choose the set of auxil...,Please see summary of weaknesses.,3 = Good: This paper is of interest to the *AC...,No,1 = No usable datasets submitted.,1 = No usable sof